In [1]:
import pyforest
data_path = '/Users/jayceepang/msse/ATOM_CODE/original_datasets/' 

In [123]:
def check_labels(datapath): 
  """a function to check that the labels are indeed what we want (a % binding or % inhibition activiy is 
  >= 50% for that molecule to be active against that NEK)"""
   assay_pct_dfs = [file for file in os.listdir(datapath) if file.startswith('NEK')]
   for file in assay_pct_dfs:
    full_file = os.path.join(datapath, file)
    df = pd.read_csv(full_file)
    
    pct_col = [col for col in df.columns if col.startswith('pct_')] 
    len(pct_col) == 1 
    pct_col = pct_col[0]
    invalid_labels = df[df['active']!= (df[pct_col]>=50).astype(int)]
    if invalid_labels.empty: 
      # print(f'correct label assignment based on {pct_col} column')
      pass
    else: 
      print(f'Error on label assignement.')
      print(invalid_labels)
    # check for duplicates here 
    # duplicates = df[df.duplicated()]
    print(f'{df.shape}')
    # duplicates = df[df.duplicated(subset=['base_rdkit_smiles'])]
    duplicates = df.duplicated()
    # duplicates = df[df.duplicated(subset=['base_rdkit_smiles'])]
    if duplicates.any():
        print('Duplicate rows found:')
        print(duplicates)
        df_cleaned = df.drop_duplicates()
        print(f'df size original: {df.shape}')
        print(f'remove duplicates. {df_cleaned.shape}')
check_labels(data_path)

IndentationError: unexpected indent (959015697.py, line 4)

In [126]:
# must specify filename because the 50pct binding and inhibition files differ
# to-do: discuss how to do this --> this will create a file that is just this but with column specifying split
#       do we want to already integrate a split_uuid like AMPL? 
#       additionally, we still have to use the splits to create the "scaled df" 
def remove_duplicates(datapath, filename):
    df = pd.read_csv(f'{datapath}{filename}')
    duplicates = df.duplicated(subset=['base_rdkit_smiles'])
    if duplicates.any():
        print('Duplicate rows found:')
        print(df[duplicates])
        df_cleaned = df.drop_duplicates(subset=['base_rdkit_smiles'])
        print(f'df size original: {df.shape}')
        print(f'Removed duplicates. New df size: {df_cleaned.shape}')
    else: 
        df_cleaned = df
    return df_cleaned
    
def split_data(datapath, filename, train_ratio=.8, test_ratio=0.2): 
    """filename: NEK#_1_uM_min_50_pct_(binding or inhibition).csv"""
    df = remove_duplicates(datapath, filename)

    # is this extra/too much and should we just assume what labels coincide with majority and minority? 
    # determine majority/minority class 
    #       AMPL takes in 'response column' 
    class_labels = df['active'].value_counts() 
    print(class_labels)
    if len(class_labels)>1: 
        majority_class_label =class_labels.idxmax() 
        majority_num = class_labels.max() 
        minority_class_label = class_labels.idxmin()
        minority_count = class_labels.min()
    df_majority = df[df['active']==majority_class_label]
    df_minority=df[df['active']==minority_class_label]
    n = round(1/test_ratio) # how else can we do this? i think we should keep kfold splits 
            # 1/.2 = 5 splits 
    kf = KFold(n_splits=n,shuffle=True, random_state=42)
    # majority 
    for i, (_, v_ind) in enumerate(kf.split(df_majority)):
        df_majority.loc[df_majority.index[v_ind], 'fold'] = f"fold{i+1}"
    # minority 
    for i, (_, v_ind) in enumerate(kf.split(df_minority)):
        df_minority.loc[df_minority.index[v_ind], 'fold'] = f"fold{i+1}"
    all_fold_df = pd.concat([df_majority,df_minority])

    print(all_fold_df.shape)
    print(all_fold_df.active.value_counts())
    ## actually, this might be perfect jsut to save as "split df" because then 
    # in the next step of creating a dataset, you can choose what fold you want to be the test set 
    return all_fold_df
        
    

# check for duplicates and split

In [127]:
assay_pct_dfs = [file for file in os.listdir(data_path) if file.startswith('NEK')]
print(assay_pct_dfs)
for i in range(len(assay_pct_dfs)): 
    cleaned_split_df = split_data(data_path, assay_pct_dfs[i]) # probably save to csv 
    

<IPython.core.display.Javascript object>

['NEK9_1_uM_min_50_pct_inhibition.csv', 'NEK2_1_uM_min_50_pct_inhibition.csv', 'NEK5_1_uM_min_50_pct_binding.csv', 'NEK9_1_uM_min_50_pct_binding.csv', 'NEK3_1_uM_min_50_pct_binding.csv', 'NEK2_1_uM_min_50_pct_binding.csv']


<IPython.core.display.Javascript object>

active
0    351
1     42
Name: count, dtype: int64


<IPython.core.display.Javascript object>

...

active
0    1351
1      57
Name: count, dtype: int64


<IPython.core.display.Javascript object>

/var/folders/wb/cvnnx9qn3tj18cq5_9wx39xm0000gn/T/ipykernel_34654/1574895176.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_majority.loc[df_majority.index[v_ind], 'fold'] = f"fold{i+1}"
/var/folders/wb/cvnnx9qn3tj18cq5_9wx39xm0000gn/T/ipykernel_34654/1574895176.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_minority.loc[df_minority.index[v_ind], 'fold'] = f"fold{i+1}"


<IPython.core.display.Javascript object>

(1408, 6)
active
0    1351
1      57
Name: count, dtype: int64


In [104]:
# I'm checking the scaled_descriptors files because I knew there were duplicates in these files 
desc_dir = '/Users/jayceepang/msse/ATOM_CODE/original_datasets/scaled_descriptors/'
desc_dfs = [file for file in os.listdir(desc_dir) if file.startswith('NEK')]

print(desc_dfs)
for i in range(len(desc_dfs)): 
    split_data(desc_dir, desc_dfs[i])

<IPython.core.display.Javascript object>

['NEK5_1_uM_min_50_pct_binding_with_moe_descriptors.csv', 'NEK3_1_uM_min_50_pct_binding_with_moe_descriptors.csv', 'NEK9_1_uM_min_50_pct_binding_with_moe_descriptors.csv', 'NEK2_1_uM_min_50_pct_binding_with_moe_descriptors.csv', 'NEK2_1_uM_min_50_pct_inhibition_with_moe_descriptors.csv', 'NEK9_1_uM_min_50_pct_inhibition_with_moe_descriptors.csv']


<IPython.core.display.Javascript object>

(1237, 309)
active
0    1140
1      97
Name: count, dtype: int64


<IPython.core.display.Javascript object>

...

(1408, 309)
active
0    1351
1      57
Name: count, dtype: int64


<IPython.core.display.Javascript object>

(2044, 309)
active
0    1904
1     140
Name: count, dtype: int64
Duplicate rows found:
     compound_id                                  base_rdkit_smiles  active  \
895     kdb_1014          Nc1nccc(-c2c(-c3ccc(F)cc3)ncn2C2CCNCC2)n1       0   
1056    kdb_3409  [O-][S+]1CCN(Cc2ccc(-c3ccc4ncnc(Nc5ccc(OCc6ccc...       0   
1060    kdb_3151  CCn1c(-c2nonc2N)nc2cnc(Oc3cccc(NC(=O)c4ccc(OCC...       0   
1142    kdb_3359  CN1CCC(C(=O)Nc2n[nH]c3nc(-c4ccc(O)cc4)c(Br)cc2...       0   
1357    kdb_3412  COc1ccc2cc(-c3c(-c4ccncc4)nc(-c4ccc([S+](C)[O-...       0   
1449    kdb_3208  COc1ccc2cc(-c3c(-c4ccncc4)nc(-c4ccc([S+](C)[O-...       0   
1561    kdb_3362  CCN1CCC(CC(=O)Nc2n[nH]c3nc(-c4cccs4)c(Br)cc23)CC1       0   
1694    kdb_2601                Fc1ccc(-c2nc3n(c2-c2ccncc2)CCS3)cc1       0   
1760    kdb_2432  Nc1ncnc2occ(-c3ccc(NC(=O)Nc4cc(C(F)(F)F)ccc4F)...       0   
1763    kdb_2231  [O-][S+]1CCN(Cc2ccc(-c3ccc4ncnc(Nc5ccc(OCc6ccc...       0   
1911    kdb_3411  [O-][S+]1CCN(Cc2ccc(-c3ccc

<IPython.core.display.Javascript object>

(393, 309)
active
0    351
1     42
Name: count, dtype: int64


In [60]:
1904+140=2044

2044